# Úkol č. 1 - vizualizace dat a web scraping (do 20. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.volby.cz týkající se voleb do zastupitelstva Vámi vybraného (většího) města, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Výběr zdroje dat

Vyberte si větší město, které má zastupitelstvo druhu 3 (Zastupitelstvo statutárního města) a strojově stáhněte informace o stranách a kandidátkách z následujících let:
 * [2002](https://www.volby.cz/pls/kv2002/kv12?xjazyk=CZ&xid=0), [2006](https://www.volby.cz/pls/kv2006/kv12?xjazyk=CZ&xid=0), [2010](https://www.volby.cz/pls/kv2010/kv12?xjazyk=CZ&xid=0), [2014](https://www.volby.cz/pls/kv2014/kv12?xjazyk=CZ&xid=0) a [2018](https://www.volby.cz/pls/kv2018/kv12?xjazyk=CZ&xid=0).
 

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data pro vybrané město a uložte je všechny do (asi dvou) přehledných tabulek ve formátu _csv_.
  * Data musí obsahovat _alespoň_ toto:
    * Vývoj výsledků (v procentech i počtu hlasů) pro jednotlivé strany v jednotlivých letech.
    * Seznam všech kandidátů všech stran v jednotlivých letech, u kandidáta by mělo být zaznamenáno: jméno, věk v době voleb, navrhující strana, politická příslušnost, volební zisk (procento i počet hlasů), pořadí na kandidátce, pořadí zvolení, jestli získal mandát (tyto informace získáte souhrnně ve _jmenných seznamech_).
  * V druhé části Vašeho Jupyter notebooku pracujte s těmito tabulkami načtenými z _csv_ souboru (aby opravující nemusel spouštět stahování z webu).
  * Tabulky ve formátu _csv_ také odevzdejte.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
    * Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
    * Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
    * Časový vývoj podílu kandidujících s titulem a bez titulu.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.
  * (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.
  * (až +2 body) Najděte nějaký balíček, který Vám dovolí do Vašeho notebooku zavést interaktivní prvky, např. si vyberete v select-boxu stranu a Váš notebook zobrazí grafy pouze pro ni atp.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) se staženými daty.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [240]:
### odtud už je to Vaše

import numpy as np
import pandas as pd
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns


pd.options.display.max_columns = None
pd.options.display.max_rows = None


# volby Liberec
url = 'https://www.volby.cz/pls/kv2018/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=5103&xobec=563889' # výsledky voleb

dfs = pd.read_html(url,flavor='html5lib')[1]
dfs.columns = ['kandid listina', 'kandid nazev', 'hlasy abs', 'hlasy procenta', 'pocet kandidatu', 'prepocet pocet kandidatu', 'prepocet procenta hlasu', 'pocet mandatu', 'podily hlasu']
dfs['podily hlasu'] = dfs['podily hlasu'].apply(lambda x: 0 if (x is np.nan or x == '-') else 1)

display(dfs)

,kandid listina,kandid nazev,hlasy abs,hlasy procenta,pocet kandidatu,prepocet pocet kandidatu,prepocet procenta hlasu,pocet mandatu,podily hlasu
0,1,Unie pro sport a zdraví,50 604,426,39,"1 188 679,00",425,0,0
1,2,Svob.a př.dem.-T.Okamura (SPD),54 539,459,39,"1 188 679,00",458,0,0
2,3,Starostové pro Liberecký kraj,381 613,3210,39,"1 188 679,00",3210,16,1
3,4,Občanská demokratická strana,99 621,838,39,"1 188 679,00",838,4,1
4,5,ANO 2011,253 959,2136,39,"1 188 679,00",2136,11,1
5,6,Liberec otevřený lidem - LOL!,62 686,527,39,"1 188 679,00",527,2,1
6,7,PRO 2016 s podporou Svobodných,42 636,359,38,"1 158 200,05",368,0,0
7,8,Liberec Plus+,18 622,157,39,"1 188 679,00",156,0,0
8,9,Komunistická str.Čech a Moravy,35 979,303,39,"1 188 679,00",302,0,0
9,10,Změna pro Liberec,157 737,1327,39,"1 188 679,00",1326,6,1


In [168]:
# 2006, 2010, 2014, 2018

url = 'https://www.volby.cz/pls/kv2010/kv21111?xjazyk=CZ&xid=1&xv=11&xdz=3&xnumnuts=5103&xobec=563889&xstrana=0' # výsledky voleb
kand_data = pd.read_html(url,flavor='html5lib')[0]
kand_data.columns = ['kandid cislo', 'kandid nazev', 'kandid poradi', 'prijmeni jmeno', 'vek', 'navrhujici strana', 'politicka prislusnost', 'hlasy abs', 'hlasy procenta', 'poradi zvoleni', 'mandat']

tmp = kand_data['prijmeni jmeno'].astype(str).str.split("\s(?=(?:\S+\.))", n = 2, expand = True)
kand_data['prijmeni jmeno'] = tmp[0]
tmp[1].fillna('', inplace=True)
tmp[2].fillna('', inplace=True)
kand_data['tituly'] = tmp[1] + ' ' + tmp[2]
kand_data['tituly'] = kand_data['tituly'].apply(lambda x: np.nan if x == ' ' else x)

kand_data['poradi zvoleni'] = kand_data['poradi zvoleni'].apply(lambda x: np.nan if x == '-' else x)
kand_data['mandat'] = kand_data['mandat'].apply(lambda x: 0 if x == '-' else 1)

display(kand_data)

,kandid cislo,kandid nazev,kandid poradi,prijmeni jmeno,vek,navrhujici strana,politicka prislusnost,hlasy abs,hlasy procenta,poradi zvoleni,mandat,tituly
0,2,Změna pro Liberec,19,Abel Daniel Tibor,46,NK,BEZPP,6 191,246,9,0,Ing.
1,11,Strana Práv Občanů ZEMANOVCI,29,Abrt Přemysl,38,SPOZ,BEZPP,209,224,NaN,0,NaN
2,2,Změna pro Liberec,2,Absolonová Kateřina,44,NK,BEZPP,8 467,337,2,1,MUDr.
3,7,Demokratická regionální strana,11,Andreasová Hana,27,DRS,BEZPP,1 140,240,NaN,0,NaN
4,9,SOS - Liberecký zastupitel,14,Antuš Jan,50,SOS,BEZPP,1 184,272,NaN,0,Mgr.
5,9,SOS - Liberecký zastupitel,35,Baba Jan,51,SOS,BEZPP,783,179,NaN,0,MVDr.
6,1,Unie pro sport a zdraví,25,Bahníková Štefania,57,USZ,USZ,2 038,240,22,0,Ing.
7,12,Liberec občanům,34,Barthová Milena,58,KDU-ČSL,BEZPP,3 396,208,28,0,NaN
8,2,Změna pro Liberec,16,Barthová Petra,39,NK,BEZPP,6 543,260,6,0,Mgr.
9,6,Věci veřejné,15,Bartoníček Miroslav,46,VV,VV,2 182,246,12,0,NaN


In [179]:
# 2002

url = 'https://www.volby.cz/pls/kv2002/kv21111?xjazyk=CZ&xid=1&xv=11&xdz=3&xnumnuts=5103&xobec=563889&xstrana=0' # výsledky voleb
kand2_data = pd.read_html(url,flavor='html5lib')[0]
kand2_data.columns = ['kandid cislo', 'kandid nazev', 'kandid poradi', 'prijmeni jmeno', 'tituly', 'vek', 'navrhujici strana', 'politicka prislusnost', 'hlasy abs', 'hlasy procenta', 'poradi zvoleni', 'mandat']

kand2_data['mandat'] = kand2_data['mandat'].apply(lambda x: 1 if x == '*' else 0)
#kand2_data['poradi zvoleni'] = kand2_data['poradi zvoleni'].apply(lambda x: np.nan if np.isnan(x) else int(x))

display(kand2_data)

,kandid cislo,kandid nazev,kandid poradi,prijmeni jmeno,tituly,vek,navrhujici strana,politicka prislusnost,hlasy abs,hlasy procenta,poradi zvoleni,mandat
0,11,Str.pro otevřenou společnost,15,Absolonová Kateřina,MUDr.,36,SOS,BEZPP,2 749,2.80,12.0,0
1,7,Naděje,20,Adam Lubomír,NaN,33,N,BEZPP,188,2.07,NaN,0
2,7,Naděje,2,Apltauer Jan,Ing.,27,N,BEZPP,487,5.38,NaN,0
3,14,Unie pro sport a zdraví,3,Bahníková Štefania,Ing.,49,USZ,USZ,2 320,3.27,4.0,0
4,14,Unie pro sport a zdraví,11,Bachtík Jan,NaN,68,USZ,USZ,1 905,2.69,9.0,0
5,13,Pravý Blok,14,Balvirčáková Blanka,NaN,20,PB,BEZPP,105,3.65,NaN,0
6,3,Demokratická regionální strana,27,Bartoš Jan,NaN,58,DRS,DRS,1 336,2.21,24.0,0
7,13,Pravý Blok,12,Bartoš Petr,NaN,54,PB,BEZPP,114,3.96,NaN,0
8,5,Křesť.demokr.unie-Čs.str.lid.,31,Bartošová Irena,Ing.,46,KDU-ČSL,KDU-ČSL,739,2.08,NaN,0
9,13,Pravý Blok,10,Bartošová Sylvie,NaN,47,PB,BEZPP,118,4.10,NaN,0


In [241]:
# TODO: save to csv


In [243]:
# TODO:

# - Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
# - Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
# - Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
# - Časový vývoj podílu kandidujících s titulem a bez titulu.

In [ ]:
# TODO +2

# (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.

In [ ]:
# TODO +4

# (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.